### My dependencies

In [1]:
# dependacies
import os
import numpy as np
# import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
# from geopy.exc import GeocoderTimedOut
# from geopy.geocoders import Nominatim
# from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go

from sklearn.impute import KNNImputer

datafolder = 'data/'

max_data = datetime(2023, 3, 31, 23, 59, 59)


### setup basis data

#### 1. Disruption

In [2]:
disruptions_2023 = pd.read_csv(datafolder + 'disruptions-2023.csv', sep=',', low_memory=False, encoding='utf-8')

disruptions_2023['rdt_lines'] = disruptions_2023['rdt_lines'].str.lower()
disruptions_2023['rdt_station_names'] = disruptions_2023['rdt_station_names'].str.lower()

# Convert the 'start_time' and 'end_time' column to datetime
disruptions_2023['start_time'] = pd.to_datetime(disruptions_2023['start_time'], format="%Y-%m-%d %H:%M:%S")
disruptions_2023['end_time'] = pd.to_datetime(disruptions_2023['end_time'], format="%Y-%m-%d %H:%M:%S")
# Filter the DataFrame to end of march
disruptions_2023 = disruptions_2023[disruptions_2023['start_time'] < max_data]

# english
disruptions_2023 = disruptions_2023.iloc[:,[2,4,9,10,11,12,13]]

# disruptions_2023 = pd.concat([disruptions_2023, pd.get_dummies(disruptions_2023['statistical_cause_en'])], axis=1)

display(disruptions_2023)

# create lose lanes so it is easyer to calculate and seporate
lose_disruptions_2023 = disruptions_2023.copy()

lose_disruptions_2023['rdt_lines'] = lose_disruptions_2023['rdt_lines'].fillna('')
lose_disruptions_2023['rdt_lines'] = lose_disruptions_2023['rdt_lines'].str.split(',')

lose_disruptions_with_station_names = lose_disruptions_2023.explode('rdt_lines').reset_index(drop=True)

# this is the one without station names so i drop it.
lose_disruptions_2023 = lose_disruptions_with_station_names.drop(columns=['rdt_station_names'])

# now to create the one with the station names lose from the lines
lose_disruptions_with_station_names['rdt_station_names'] = lose_disruptions_with_station_names['rdt_station_names'].str.split(',')

lose_disruptions_with_station_names = lose_disruptions_with_station_names.explode('rdt_station_names').reset_index(drop=True)

# display the lose data
display(lose_disruptions_2023)
display(lose_disruptions_with_station_names)

,rdt_lines,rdt_station_names,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
0,"amsterdam centraal - schiphol airport, rotterd...","amsterdam centraal,amsterdam lelylaan,amsterda...",points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
1,leeuwarden - zwolle,"heerenveen,wolvega,heerenveen ijsstadion",an animal on the railway track,external,2023-01-01 10:31:49,2023-01-01 10:56:17,24.0
2,aachen hbf - heerlen,"aachen hbf,eygelshoven markt,heerlen,heerlen d...",problems with the rolling stock,rolling stock,2023-01-01 13:19:24,2023-01-02 00:02:39,643.0
3,winterswijk - zutphen,"vorden,zutphen",collision,accidents,2023-01-01 17:15:22,2023-01-01 20:14:23,179.0
4,aachen hbf - heerlen,"aachen hbf,eygelshoven markt,heerlen,heerlen d...",problems with the rolling stock,rolling stock,2023-01-02 05:57:27,2023-01-03 02:07:13,1210.0
...,...,...,...,...,...,...,...
1218,breda - roosendaal,"breda,etten-leur,roosendaal",broken down train,rolling stock,2023-03-31 10:44:58,2023-03-31 10:55:53,11.0
1219,leiden centraal - schiphol airport,"hoofddorp,leiden centraal,nieuw vennep,schipho...",broken down train,rolling stock,2023-03-31 12:21:49,2023-03-31 12:27:27,6.0
1220,"arnhem centraal - basel sbb, arnhem centraal -...","arnhem centraal,arnhem velperpoort,duiven,emme...",broken down train,rolling stock,2023-03-31 17:48:22,2023-03-31 18:17:16,29.0
1221,leiden centraal - schiphol airport,"hoofddorp,schiphol airport",broken down train,rolling stock,2023-03-31 19:17:20,2023-03-31 19:27:58,11.0


,rdt_lines,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
0,amsterdam centraal - schiphol airport,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
1,rotterdam centraal - schiphol airport (hsl),points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
2,leeuwarden - zwolle,an animal on the railway track,external,2023-01-01 10:31:49,2023-01-01 10:56:17,24.0
3,aachen hbf - heerlen,problems with the rolling stock,rolling stock,2023-01-01 13:19:24,2023-01-02 00:02:39,643.0
4,winterswijk - zutphen,collision,accidents,2023-01-01 17:15:22,2023-01-01 20:14:23,179.0
...,...,...,...,...,...,...
1961,arnhem centraal - basel sbb,broken down train,rolling stock,2023-03-31 17:48:22,2023-03-31 18:17:16,29.0
1962,arnhem centraal - düsseldorf hbf,broken down train,rolling stock,2023-03-31 17:48:22,2023-03-31 18:17:16,29.0
1963,arnhem centraal - frankfurt (m) hbf,broken down train,rolling stock,2023-03-31 17:48:22,2023-03-31 18:17:16,29.0
1964,leiden centraal - schiphol airport,broken down train,rolling stock,2023-03-31 19:17:20,2023-03-31 19:27:58,11.0


,rdt_lines,rdt_station_names,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
0,amsterdam centraal - schiphol airport,amsterdam centraal,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
1,amsterdam centraal - schiphol airport,amsterdam lelylaan,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
2,amsterdam centraal - schiphol airport,amsterdam sloterdijk,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
3,amsterdam centraal - schiphol airport,rotterdam centraal,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
4,amsterdam centraal - schiphol airport,schiphol airport,points failure,infrastructure,2023-01-01 08:19:26,2023-01-01 22:43:08,864.0
...,...,...,...,...,...,...,...
10776,leiden centraal - schiphol airport,hoofddorp,broken down train,rolling stock,2023-03-31 19:17:20,2023-03-31 19:27:58,11.0
10777,leiden centraal - schiphol airport,schiphol airport,broken down train,rolling stock,2023-03-31 19:17:20,2023-03-31 19:27:58,11.0
10778,arnhem centraal - zutphen,brummen,broken down train,rolling stock,2023-03-31 19:21:04,2023-03-31 19:48:51,28.0
10779,arnhem centraal - zutphen,dieren,broken down train,rolling stock,2023-03-31 19:21:04,2023-03-31 19:48:51,28.0


#### 2. weather

In [3]:
weather_2023 = pd.read_csv(datafolder + '2023-jan-mar.csv', sep=',', low_memory=False, encoding='utf-8', skiprows=80)

display(weather_2023)

weather_2023['start_time'] = weather_2023['start_time'].astype(str) + ' ' + (weather_2023['HH'] - 1).astype(str) + ':00:00'

weather_2023['start_time'] = pd.to_datetime(weather_2023['start_time'], format="%Y%m%d %H:%M:%S")
#  drop columns that are not needed
# weather_2023 = weather_2023.drop(columns=['HH', 'WW', 'IX', 'FH', 'FX', 'TD', 'SQ', 'RH'])
weather_2023 = weather_2023.drop(columns=['HH'])

display(weather_2023.head(12))

,STN,start_time,HH,DD,FH,FF,FX,temp,T10N,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
0,209,20230101,1,220.0,140.0,150.0,170.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
1,209,20230101,2,220.0,130.0,130.0,170.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
2,209,20230101,3,240.0,140.0,130.0,200.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
3,209,20230101,4,220.0,120.0,130.0,170.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
4,209,20230101,5,230.0,130.0,140.0,170.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99355,391,20230331,20,210.0,50.0,40.0,120.0,99.0,NaN,88.0,...,NaN,NaN,93.0,NaN,6,NaN,NaN,NaN,NaN,NaN
99356,391,20230331,21,220.0,40.0,50.0,100.0,99.0,NaN,86.0,...,NaN,NaN,91.0,NaN,6,NaN,NaN,NaN,NaN,NaN
99357,391,20230331,22,210.0,40.0,40.0,110.0,97.0,NaN,87.0,...,NaN,NaN,93.0,NaN,6,NaN,NaN,NaN,NaN,NaN
99358,391,20230331,23,220.0,40.0,40.0,90.0,95.0,NaN,88.0,...,NaN,NaN,95.0,NaN,6,NaN,NaN,NaN,NaN,NaN


,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,VV,N,U,WW,IX,M,R,S,O,Y
0,209,2023-01-01 00:00:00,220.0,140.0,150.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
1,209,2023-01-01 01:00:00,220.0,130.0,130.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
2,209,2023-01-01 02:00:00,240.0,140.0,130.0,200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
3,209,2023-01-01 03:00:00,220.0,120.0,130.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
4,209,2023-01-01 04:00:00,230.0,130.0,140.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
5,209,2023-01-01 05:00:00,220.0,130.0,130.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
6,209,2023-01-01 06:00:00,220.0,130.0,120.0,160.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
7,209,2023-01-01 07:00:00,230.0,120.0,110.0,160.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
8,209,2023-01-01 08:00:00,220.0,110.0,110.0,130.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
9,209,2023-01-01 09:00:00,220.0,110.0,110.0,140.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN


#### 3. weather station locations

In [4]:
weather_station_location = pd.read_csv(datafolder + 'Weather-stations-loc.csv', sep=',', low_memory=False, encoding='utf-8')
display(weather_station_location)

,STN,LON(east),LAT(north),ALT(m),NAME
0,209,4.518,52.465,0.0,IJmond
1,210,4.430,52.171,-0.2,ValkenburgZh
2,215,4.437,52.141,-1.1,Voorschoten
3,225,4.555,52.463,4.4,IJmuiden
4,235,4.781,52.928,1.2,DeKooy
5,240,4.790,52.318,-3.3,Schiphol
6,242,4.921,53.241,10.8,Vlieland
7,248,5.174,52.634,0.8,Wijdenes
8,249,4.979,52.644,-2.4,Berkhout
9,251,5.346,53.392,0.7,HoornTerschelling


#### 4. all train data

In [5]:
Train_trafic_2023 = pd.read_csv(datafolder + 'services-2023/services-2023.csv', sep=',', low_memory=False)

Train_trafic_2023['Service:Date'] = pd.to_datetime(Train_trafic_2023['Service:Date'], format="%Y-%m-%d")

Train_trafic_2023['Service:Type'] = Train_trafic_2023['Service:Type'].str.lower()
Train_trafic_2023['Service:Company'] = Train_trafic_2023['Service:Company'].str.lower()

# Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Company'] == 'ns']

Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Date'] < max_data]

display(Train_trafic_2023.sort_values(by='Service:Date'))

,Service:RDT-ID,Service:Date,Service:Type,Service:Company,Service:Train number,Service:Completely cancelled,Service:Partly cancelled,Service:Maximum delay,Stop:RDT-ID,Stop:Station code,Stop:Station name,Stop:Arrival time,Stop:Arrival delay,Stop:Arrival cancelled,Stop:Departure time,Stop:Departure delay,Stop:Departure cancelled
0,10367702,2023-01-01,intercity,ns,1410,False,False,3,93067918,RTD,Rotterdam Centraal,NaN,NaN,NaN,2023-01-01T02:00:00+01:00,1.0,False
25314,10370347,2023-01-01,sprinter,ns,5867,False,False,4,93094279,ASD,Amsterdam Centraal,NaN,NaN,NaN,2023-01-01T18:41:00+01:00,1.0,False
25315,10370347,2023-01-01,sprinter,ns,5867,False,False,0,93094280,ASDM,Amsterdam Muiderpoort,2023-01-01T18:46:00+01:00,1.0,False,2023-01-01T18:46:00+01:00,1.0,False
25316,10370347,2023-01-01,sprinter,ns,5867,False,False,0,93094281,ASSP,Amsterdam Science Park,2023-01-01T18:49:00+01:00,1.0,False,2023-01-01T18:49:00+01:00,1.0,False
25317,10370347,2023-01-01,sprinter,ns,5867,False,False,0,93094282,DMN,Diemen,2023-01-01T18:52:00+01:00,1.0,False,2023-01-01T18:52:00+01:00,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098692,10911605,2023-03-31,snelbus i.p.v. trein,ns,983199,True,True,0,98171415,DB,Driebergen-Zeist,2023-03-31T12:15:00+02:00,0.0,True,NaN,NaN,NaN
5098693,10911606,2023-03-31,stopbus i.p.v. trein,valleilijn,984414,True,True,0,98171416,ED,Ede-Wageningen,NaN,NaN,NaN,2023-03-31T11:30:00+02:00,0.0,True
5098694,10911606,2023-03-31,stopbus i.p.v. trein,valleilijn,984414,True,True,0,98171417,EDC,Ede Centrum,2023-03-31T11:35:00+02:00,0.0,True,2023-03-31T11:35:00+02:00,0.0,True
5098681,10911602,2023-03-31,intercity,ns,3034,False,False,0,98171404,AMRN,Alkmaar Noord,2023-03-31T12:52:00+02:00,1.0,False,2023-03-31T12:52:00+02:00,1.0,False


# Nul chacking and fixing

## checking for null

#### 1. disruptions
In diruptions there are little fields not filled in and seeing that that would deliver little inpact, i think the data can be kept for later use.

In [6]:
display(disruptions_2023.isna().sum())
display(lose_disruptions_2023.isna().sum())
display(lose_disruptions_with_station_names.isna().sum())

rdt_lines               2
rdt_station_names       2
statistical_cause_en    0
cause_group             0
start_time              0
end_time                2
duration_minutes        2
dtype: int64

rdt_lines               0
statistical_cause_en    0
cause_group             0
start_time              0
end_time                2
duration_minutes        2
dtype: int64

rdt_lines               0
rdt_station_names       2
statistical_cause_en    0
cause_group             0
start_time              0
end_time                6
duration_minutes        6
dtype: int64

#### 2. weather

in the weather dataset it can be said that some record points don't have surtan sensors and that the data then is missing. I will see if i can work around that later but for now i will use that data.

In [7]:
display(weather_2023.isna().sum())

STN               0
start_time        0
DD             2499
FH             2500
FF             2499
FX             2500
temp          25920
T10N          87120
TD            25920
SQ            30240
Q             30240
DR            28081
RH            28080
P             47520
VV            45361
N             56160
U             25920
WW            73164
IX                0
M             45415
R             45415
S             45415
O             45415
Y             45415
dtype: int64

3. weather stations

In [8]:
display(weather_station_location.isna().sum())

STN           0
LON(east)     0
LAT(north)    0
ALT(m)        0
NAME          0
dtype: int64

#### 4. all train data

In [9]:
Train_trafic_2023 = Train_trafic_2023.drop(columns=['Stop:RDT-ID'])

display(Train_trafic_2023.isna().sum())
display(Train_trafic_2023.dtypes)

Service:RDT-ID                       0
Service:Date                         0
Service:Type                         0
Service:Company                      0
Service:Train number                 0
Service:Completely cancelled         0
Service:Partly cancelled             0
Service:Maximum delay                0
Stop:Station code                 6632
Stop:Station name                    0
Stop:Arrival time               551049
Stop:Arrival delay              551049
Stop:Arrival cancelled          551049
Stop:Departure time             550943
Stop:Departure delay            550943
Stop:Departure cancelled        550943
dtype: int64

Service:RDT-ID                           int64
Service:Date                    datetime64[ns]
Service:Type                            object
Service:Company                         object
Service:Train number                     int64
Service:Completely cancelled              bool
Service:Partly cancelled                  bool
Service:Maximum delay                    int64
Stop:Station code                       object
Stop:Station name                       object
Stop:Arrival time                       object
Stop:Arrival delay                     float64
Stop:Arrival cancelled                  object
Stop:Departure time                     object
Stop:Departure delay                   float64
Stop:Departure cancelled                object
dtype: object

# remove nulls

#### 1. disruptions

In [ ]:
display(disruptions_2023[disruptions_2023['rdt_lines'].isna()])
display(disruptions_2023[disruptions_2023['end_time'].isna()])
display(disruptions_2023.isna().sum())

In [ ]:
# show a pie graph to see what is most caused.
fig = px.pie(disruptions_2023['cause_group'].value_counts(), values='count', names=disruptions_2023['cause_group'].value_counts().index, title='Distribution of Causes')
fig.show()

big_fig = go.Figure()

for cause in disruptions_2023['cause_group'].unique():
    df_cause = lose_disruptions_with_station_names[lose_disruptions_with_station_names['cause_group'] == cause]
    fig = px.histogram(df_cause, x='duration_minutes')
    fig.update_layout(title_text='Time to Fix for ' + cause)
    fig.show()

    big_fig.add_trace(go.Histogram(x=df_cause['duration_minutes'], name=cause, opacity=0.3))

    most_common_names = df_cause['rdt_station_names'].value_counts()

    fig = px.bar(most_common_names, x=most_common_names.index, y='count', title='Most Common train station for ' + cause)

    fig.show()

big_fig.show()

In [ ]:
#select rows to train and what I want predicted.
'''
features = ['start_time', 'end_time']
target = 'duration_minutes'

X = disruptions_2023.dropna()[features]

X['start_time'] = pd.to_datetime(X['start_time']).astype(np.int64) // 10**9
X['end_time'] = pd.to_datetime(X['end_time']).astype(np.int64) // 10**9

y = disruptions_2023.dropna()[target]

# split into trainign data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
print('There are in total', len(X), 'observations, of which', len(X_train), 'are now in the train set, and', len(X_test), 'in the test set.')

# model the linear regression.
from sklearn.linear_model import LinearRegression
model = LinearRegression()
result = model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("R²:", score)

display(disruptions_2023.isna().sum())
'''

#### 2 weather

In [ ]:
# create heatmap
plt.figure(figsize=(15,15))
mask = np.tril(weather_2023.corr())
sns.heatmap(weather_2023.corr(), cmap="coolwarm", annot=True, fmt=".1f", mask=mask, linewidths=.75, linecolor='silver')

decision tree

In [ ]:
#select rows to train and what I want predicted.
features = ['STN','start_time','FH','FF','FX', 'T10N', 'TD']
target = 'temp'

X = weather_2023.dropna()[features]

X['start_time'] = pd.to_datetime(X['start_time']).astype(np.int64) // 10**9

y = weather_2023.dropna()[target]

# split into trainign data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
print('There are in total', len(X), 'observations, of which', len(X_train), 'are now in the train set, and', len(X_test), 'in the test set.')

# model the linear regression.
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("Accuracy:", score)

display(weather_2023.isna().sum())

In [ ]:
merged_weather_station = pd.merge(weather_2023, weather_station_location, on='STN')

display(merged_weather_station.sample(6))

# Create the figure
fig = go.Figure(go.Densitymapbox(lat=merged_weather_station['LAT(north)'], lon=merged_weather_station['LON(east)'], z=merged_weather_station['temp'], radius=5))

# Update layout
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=merged_weather_station['LON(east)'].mean(), mapbox_center_lat=merged_weather_station['LAT(north)'].mean(), mapbox=dict(zoom=5))



# Show the figure
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dcc.Graph(id='temperature-map'),
    dcc.Dropdown(
        id='datetime-picker',
        options=[{'label': str(dt), 'value': dt} for dt in merged_weather_station['start_time']],
        value=merged_weather_station['start_time'].iloc[0]  # Set initial value
    )
])

# Define callback to update the graph based on selected datetime
@app.callback(
    Output('temperature-map', 'figure'),
    [Input('datetime-picker', 'value')]
)
def update_map(selected_datetime):
    filtered_df = merged_weather_station[merged_weather_station['start_time'] == selected_datetime]

    fig = go.Figure(go.Densitymapbox(lat=filtered_df['LAT(north)'], lon=filtered_df['LON(east)'], z=filtered_df['temp'] / 10))
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center_lon=filtered_df['LON(east)'].mean(),
        mapbox_center_lat=filtered_df['LAT(north)'].mean(),
        mapbox=dict(zoom=5),
        title=f"Hourly Temperature Density Map ({selected_datetime})",
        xaxis_title="Longitude",
        yaxis_title="Latitude"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
fig = px.bar(x=features, y=model.feature_importances_)
fig.show()

#### 4. train data

In [ ]:
# display(Train_trafic_2023.isna().sum())
fig = px.bar(x=Train_trafic_2023['Service:Type'].value_counts().index, y=Train_trafic_2023['Service:Type'].value_counts())
fig.show()

fig = px.bar(x=Train_trafic_2023['Stop:Station name'].value_counts().index, y=Train_trafic_2023['Stop:Station name'].value_counts())
fig.show()

In [ ]:

# create heatmap
plt.figure(figsize=(6,6))
correlation_matrix = Train_trafic_2023.select_dtypes(include=["int64", "float64"]).corr() # Train_trafic_2023.select_dtypes(include=["int64", "float64"]).corr()
mask = np.tril(correlation_matrix)
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=True, fmt=".1f", mask=mask, linewidths=.75, linecolor='silver')

In [ ]:
fig = px.bar(Train_trafic_2023, x='Stop:Station name', y='Stop:Station name', color='Service:Type')
fig.show()

In [ ]:
Train_trafic_2023['state'] = pd.Categorical(
    Train_trafic_2023[['Service:Partly cancelled', 'Service:Completely cancelled']].apply(lambda x: 'both partly and fully' if x[0] and x[1] else ('Partly cancelled' if x[0] else ('Completely cancelled' if x[1] else 'Neither partly and fully')), axis=1),
    categories=['Neither partly and fully', 'Partly cancelled', 'Completely cancelled', 'both partly and fully'],
    ordered=True
)

counts = Train_trafic_2023['state'].value_counts(sort=False)
fig = px.pie(values=counts, names=counts.index, title='Pie chart')
fig.show()

In [ ]:
# Train_trafic_2023['binned'] = pd.cut(Train_trafic_2023['Stop:Departure delay'], bins=range(0, Train_trafic_2023['Stop:Departure delay'].max() + 10, 10), include_lowest=True)
bins = list(range(0, 10, 2)) + list(range(10, 70, 10)) + list(range(70, int(Train_trafic_2023['Stop:Departure delay'].max()) +100, 100))

display(bins)

# Define your custom labels
labels = [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]

display(labels)

Train_trafic_2023['binned'] = pd.cut(Train_trafic_2023['Stop:Departure delay'], bins=bins, labels=labels, include_lowest=True)
Train_trafic_2023['binned'] = Train_trafic_2023['binned'].astype(str)

counts = Train_trafic_2023['binned'].value_counts(sort=False)
fig = px.pie(values=counts, names=counts.index, title='delay in minutes', sort=False)
fig.show()

# machine learning

#### 1. disruptions

splittig: here i split the data in training and 

In [ ]:
display(disruptions_2023.isna().sum())


features = ["start_time", "cause_group", "statistical_cause_en", "rdt_lines"]
target = "duration_minutes"

X = disruptions_2023[features].dropna()
y = disruptions_2023[target].dropna()

# X["start_time"] = X["start_time"] * 1000


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
print("There are in total", len(X), "observations, of which", len(X_train), "are now in the train set, and", len(X_test), "in the test set.")

Moddeling: lets moddel some modells and see what is best.

SVC:

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder

# Assuming X_train and y_train are pandas DataFrames
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train)

X_train_encoded = enc.transform(X_train)
X_test_encoded = enc.transform(X_test)
kernels = ['linear', 'poly', 'sigmoid', 'rbf']
c_ranges = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for kernel in kernels:
    for range in c_ranges:
        model = SVC(C=range, kernel=kernel)
        result = model.fit(X_train_encoded, y_train)
        score = model.score(X_test_encoded, y_test)
        print(f'Accuracy with {kernel} kernel and {range} C range: {score}')

# 🔬 Evaluation

In [ ]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test_encoded)
report = classification_report(y_test, predictions)
print(report)

# some calculations

In [ ]:
# https://stackoverflow.com/questions/73176918/how-do-i-change-value-names-to-numbers-in-dataframe
def transform_toNumber(dataset, column):
    # change ... to a number
    all_names = dataset[column].unique()
    dic = dict((v,k) for k,v in enumerate(all_names))

    dataset[column] = dataset[column].map(dic)
    return dataset


# create lose lanes so it is easyer to calculate and seporate
lose_disruptions_2023 = disruptions_2023.copy()

lose_disruptions_2023['rdt_lines'] = lose_disruptions_2023['rdt_lines'].fillna('')
lose_disruptions_2023['rdt_lines'] = lose_disruptions_2023['rdt_lines'].str.split(',')

lose_disruptions_with_station_names = lose_disruptions_2023.explode('rdt_lines').reset_index(drop=True)

# this is the one without station names so i drop it.
lose_disruptions_2023 = lose_disruptions_with_station_names.drop(columns=['rdt_station_names'])

# now to create the one with the station names lose from the lines
lose_disruptions_with_station_names['rdt_station_names'] = lose_disruptions_with_station_names['rdt_station_names'].str.split(',')

lose_disruptions_with_station_names = lose_disruptions_with_station_names.explode('rdt_station_names').reset_index(drop=True)

# display the lose data
display(lose_disruptions_2023)
display(lose_disruptions_with_station_names)




In [ ]:
# create the Numbered version of th data 
numbered_lose_disruptions_2023 = lose_disruptions_2023.copy()


# calculate everything in numbered_lose_disruptions_2023 to a number.

numbered_lose_disruptions_2023 = transform_toNumber(numbered_lose_disruptions_2023, 'statistical_cause_en')

numbered_lose_disruptions_2023 = transform_toNumber(numbered_lose_disruptions_2023, 'cause_group')
numbered_lose_disruptions_2023 = transform_toNumber(numbered_lose_disruptions_2023, 'rdt_lines')


display(numbered_lose_disruptions_2023)
# some basic visualization for numbered_lose_disruptions_2023
plt.figure(figsize=(10,10))
mask = np.tril(numbered_lose_disruptions_2023.corr())
sns.heatmap(numbered_lose_disruptions_2023.corr(), cmap="coolwarm", annot=True, fmt=".1f", mask=mask, linewidths=.75, linecolor='silver')

In [ ]:
numbered_lose_disruptions_with_station_names = lose_disruptions_with_station_names.copy()
# calculate everything in numbered_lose_disruptions_with_station_names to a number.

# english
numbered_lose_disruptions_with_station_names = transform_toNumber(numbered_lose_disruptions_with_station_names, 'statistical_cause_en')

numbered_lose_disruptions_with_station_names = transform_toNumber(numbered_lose_disruptions_with_station_names, 'cause_group')
numbered_lose_disruptions_with_station_names = transform_toNumber(numbered_lose_disruptions_with_station_names, 'rdt_lines')
numbered_lose_disruptions_with_station_names = transform_toNumber(numbered_lose_disruptions_with_station_names, 'rdt_station_names')


display(numbered_lose_disruptions_with_station_names)
# some basic visualization for numbered_lose_disruptions_2023
plt.figure(figsize=(10,10))
mask = np.tril(numbered_lose_disruptions_with_station_names.corr())
sns.heatmap(numbered_lose_disruptions_with_station_names.corr(), cmap="coolwarm", annot=True, fmt=".1f", mask=mask, linewidths=.75, linecolor='silver')

Show lates and earlyest recoderd time.

In [ ]:
# order the valeus by time
time_ordered_weather = weather_2023.sort_values(by='start_time', ascending=True)

# display the first and last 30 enteties.
show_ammount = 10
display(time_ordered_weather.head(show_ammount));
display(time_ordered_weather.tail(show_ammount));



create a output of corelations of the weather

plotting the locations of the weather stations

In [ ]:
plt.scatter(weather_station_location['LON(east)'], weather_station_location['LAT(north)'], c=weather_station_location['ALT(m)'])
plt.xlabel('LON(east)')
plt.ylabel('LAT(north)')
plt.colorbar(label='ALT(m)');

trying to see what there is of corelation between all the data

In [ ]:


weather_and_disturbances = pd.merge(lose_disruptions_with_station_names, merged_weather_station, left_on='rdt_station_names', right_on='NAME', how='outer')

display(lose_disruptions_with_station_names.sort_values(by='rdt_station_names'))
display(merged_weather_station.sort_values(by='NAME'))

# Calculate the difference between the two datetime columns
weather_and_disturbances['time_diff'] = (weather_and_disturbances['start_time_x'] - weather_and_disturbances['start_time_y']).abs()

# Convert the time difference to minutes
weather_and_disturbances['time_diff'] = weather_and_disturbances['time_diff'].dt.total_seconds() / 60

# Filter rows where the time difference is less than or equal to 30 minutes
weather_and_disturbances = weather_and_disturbances[weather_and_disturbances['time_diff'] <= 30]

# weather_and_disturbances = weather_and_disturbances.dropna(subset=['rdt_lines', 'rdt_station_names', 'STN'])

display(weather_and_disturbances.sort_values(by='time_diff'))


the corelation between the weather and the disruptions

In [ ]:
number_weather_and_disturbances = weather_and_disturbances.copy()


# english
number_weather_and_disturbances = transform_toNumber(number_weather_and_disturbances, 'statistical_cause_en')
number_weather_and_disturbances = transform_toNumber(number_weather_and_disturbances, 'cause_group')
number_weather_and_disturbances = transform_toNumber(number_weather_and_disturbances, 'rdt_lines')
number_weather_and_disturbances = transform_toNumber(number_weather_and_disturbances, 'rdt_station_names')
number_weather_and_disturbances = transform_toNumber(number_weather_and_disturbances, 'NAME')

display(number_weather_and_disturbances)

plt.figure(figsize=(20,20))
mask = np.tril(number_weather_and_disturbances.corr())
sns.heatmap(number_weather_and_disturbances.corr(), cmap="coolwarm", annot=True, fmt=".1f", mask=mask, linewidths=.75, linecolor='silver')

##### load in the coodanates of the train stations
Old:


In [ ]:
'''
# old code:
# Specify your file path
file_path_train_station_locations = 'train-station-locations.csv'

# Check if the file exists
if os.path.exists(datafolder + file_path_train_station_locations):
    # If the file exists, load the dataset
    train_station_locations = pd.read_csv(datafolder + file_path_train_station_locations)
else:
    # If the file doesn't exist, create a new DataFrame
    train_station_locations = pd.DataFrame(columns=['NAME'])
    train_station_locations['NAME'] = lose_disruptions_with_station_names['rdt_station_names'].unique()
    

train_station_locations = train_station_locations[np.invert((train_station_locations['NAME'].isna()))]

train_station_locations.set_index('NAME', inplace=True)
weather_station_location.set_index('NAME', inplace=True)

# Use combine_first() to fill null values
combined = train_station_locations.combine_first(weather_station_location)

# Find entries that exist in weather_station_location but not in train_station_locations
new_entries = weather_station_location.loc[weather_station_location.index.difference(combined.index)]

# Concatenate the new entries to the combined DataFrame
train_station_locations = pd.concat([combined, new_entries])

# Reset the index if needed
train_station_locations.reset_index(inplace=True)
weather_station_location.reset_index(inplace=True)

# train_station_locations = pd.merge(train_station_locations, weather_station_location, on='NAME', how='left') 

display(train_station_locations.isna().sum())
display(len(train_station_locations['NAME'].unique()))
display(train_station_locations)

train_station_locations.to_csv(datafolder + file_path_train_station_locations, index=False)
'''


new:

In [ ]:
europa_train_station_locations_original = pd.read_csv(datafolder + 'train_stations_europe.csv', low_memory=False, encoding='utf-8')

europa_train_station_locations_original = europa_train_station_locations_original.dropna(subset=['latitude'])

display(europa_train_station_locations_original)

europa_train_station_locations = europa_train_station_locations_original.copy()

europa_train_station_locations['country'] = pd.Categorical(europa_train_station_locations['country'], categories=['NL', 'DE', 'BE'], ordered=True)

europa_train_station_locations.sort_values('country', inplace=True)

# Convert 'country' back to a regular column (optional)
europa_train_station_locations['country'] = europa_train_station_locations['country'].astype(str)

europa_train_station_locations = europa_train_station_locations.drop_duplicates(subset='name').drop_duplicates(subset='name_norm')

local_train_station_locations = pd.DataFrame(columns=['NAME', 'latitude', 'longitude'])
local_train_station_locations['NAME'] = lose_disruptions_with_station_names['rdt_station_names'].unique()


europa_train_station_locations['name'] = europa_train_station_locations['name'].str.replace('[^a-zA-Z0-9 ]', '')
europa_train_station_locations['name_norm'] = europa_train_station_locations['name_norm'].str.replace('[^a-zA-Z0-9 ]', '')

local_train_station_locations['NAME'] = local_train_station_locations['NAME'].str.replace('[^a-zA-Z0-9 ]', '')


display(local_train_station_locations)



# First, set 'location' as the index in both dataframes for easier merging
europa_train_station_locations.set_index('name', inplace=True)
local_train_station_locations.set_index('NAME', inplace=True)

# Now, fill in the missing coordinates in df2 from df1
local_train_station_locations.update(europa_train_station_locations)

europa_train_station_locations.set_index('name_norm', inplace=True)

local_train_station_locations.update(europa_train_station_locations)

# Reset the index to revert 'location' back to a column
local_train_station_locations.reset_index(inplace=True)
europa_train_station_locations.reset_index(inplace=True)

display(local_train_station_locations)


Calculate the missing coordinates

###### Old:

In [ ]:
'''
# old coordanate fix
geolocator = Nominatim(user_agent="geoapiExercise")

def get_location_by_name(name):
    """This function returns a location as raw from an address will get passed to it."""
    try:
        location = geolocator.geocode(name + ', Nederland')
        if location is None:
            location = geolocator.geocode(name)
        return location
    except GeocoderTimedOut:
        return get_location_by_name(name)
    except Exception as e:
        print(f"An error occurred: {e}")

null_count_longatude = train_station_locations.isna().sum()['LON(east)']

display(train_station_locations.sort_values(by='LON(east)'))

if null_count_longatude > 0 :
    # display(null_count_longatude)
    
    null_count_longatude = null_count_longatude
    if null_count_longatude > 50 :
        null_count_longatude = 50
    
    to_calculate_stations = train_station_locations[train_station_locations['LON(east)'].isna()].head(null_count_longatude).reset_index(drop=True)
    
    to_calculate_stations = to_calculate_stations[np.invert((to_calculate_stations['NAME'].isna()))]
    
    temp_result = to_calculate_stations['NAME'].apply(lambda x: get_location_by_name(x))
    
    returned_values = pd.DataFrame(temp_result.tolist(), columns=['Address', 'location'])
    
    # Assuming 'df' is your DataFrame and 'column_to_explode' is the column you want to explode
    list_df = pd.DataFrame(returned_values['location'].to_list(), index=returned_values.index, columns=['LAT(north)', 'LON(east)'])

    # Concatenate the new DataFrame with the original DataFrame
    returned_values = pd.concat([returned_values, list_df], axis=1)

    # Now, you can drop the original column if you want
    returned_values = returned_values.drop(columns=['location'])
    
    returned_values['NAME'] = to_calculate_stations['NAME']
    display(returned_values)
    
    train_station_locations.set_index('NAME', inplace=True)
    returned_values.set_index('NAME', inplace=True)

    # Use combine_first() to fill null values
    combined = train_station_locations.combine_first(returned_values)

    # Find entries that exist in returned_values but not in train_station_locations
    new_entries = returned_values.loc[returned_values.index.difference(combined.index)]

    # Concatenate the new entries to the combined DataFrame
    train_station_locations = pd.concat([combined, new_entries])

    train_station_locations = train_station_locations.drop(columns=['Address'])
    # Reset the index if needed
    train_station_locations.reset_index(inplace=True)
    returned_values.reset_index(inplace=True)

display(train_station_locations)

train_station_locations.to_csv(datafolder + file_path_train_station_locations, index=False)
'''

###### New:

In [ ]:

display(local_train_station_locations[local_train_station_locations['latitude'].isna()])

display(local_train_station_locations.isna().sum())

fig = px.scatter_mapbox(local_train_station_locations.dropna(), lat="latitude", lon="longitude", hover_name="NAME", color_discrete_sequence=["fuchsia"], zoom=3, height=300, width=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

'''
fig = px.scatter_mapbox(europa_train_station_locations_original, lat="latitude", lon="longitude", hover_name="name_norm", color_discrete_sequence=["fuchsia"], zoom=3, height=300, width=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
'''



show most common station names

In [ ]:

most_common_names = lose_disruptions_with_station_names['rdt_station_names'].value_counts()

fig = px.bar(most_common_names, x=most_common_names.index, y='count', title='Most Common train station')

fig.show()